Coursework  
Vladimir Galkin  
Task 2   

In [3]:
# imports
import random
import math
import heapq
from collections import defaultdict

In [4]:
# global variables
SIM_TIME = 3600.0 #1h
random.seed(42)


In [5]:
# distributions
def uniform_interval():
    return random.uniform(0.5, 5/6)
def uniform_service():
    return random.uniform(1.0, 5.0)
def exp_interarrival(lmbd):
    return random.expovariate(lmbd)
def exp_service(mean):
    return random.expovariate(1.0 / mean)

In [ ]:
# sim
def simulate(arrival_func, service_func, sim_time=SIM_TIME):
    time = 0.0
    busy = 0
    departures = [] # min heapp of finish times

    arrivals = 0
    served = 0
    rejected = 0

    state_time = defaultdict(float)
    last_event_time = 0.0

    next_arrival = arrival_func()

    while time < sim_time:
        next_departure = departures[0] if departures else math.inf

        if next_arrival <= next_departure:
            time = next_arrival
            state_time[busy] += time - last_event_time
            last_event_time = time

            arrivals += 1
            if busy < 3:
                busy += 1
                finish = time + service_func()
                heapq.heappush(departures, finish)
            else:
                rejected += 1

            next_arrival = time + arrival_func()
        else:
            time = next_departure
            state_time[busy] += time - last_event_time
            last_event_time = time 

            busy -= 1
            served += 1
            heapq.heappop(departures)

    total_time = sum(state_time.values())
    P = {k: state_time[k]/total_time for k in range(4)} #probability of system being free

    Q = served/arrivals if arrivals else 0 # bandwidth
    S = served/sim_time # absolute bandwidth
    P_reject = rejected/arrivals if arrivals else 0 # probability of rejection
    K = sum(k*P[k] for k in range(4)) # aaaverage busy servers

    return {
        'P0': P[0], 'P1': P[1], 'P2': P[2], 'P3':P[3],
          'Q':Q, 'S': S, 'P_reject':P_reject, 'K': K,
            'arrivals': arrivals, 'served':served, 'rejected': rejected
    }


In [7]:
# main cw experiment
print("\nUniform Distribution")
uniform_results = simulate(uniform_interval, uniform_service)
for k,v in uniform_results.items():
    print(f"{k}: {v}")

print("\nExponential Distribution")
exp_results = simulate(lambda: exp_interarrival(1.5), lambda: exp_service(2.0))
for k,v in exp_results.items():
    print(f"{k}: {v}")


Uniform Distribution
P0: 0.002166987180934683
P1: 0.052625795478259976
P2: 0.3146617901586978
P3: 0.6305454271821076
Q: 0.5686238192257825
S: 0.8527777777777777
P_reject: 0.43082052231894796
K: 2.573585657341978
arrivals: 5399
served: 3070
rejected: 2326

Exponential Distribution
P0: 0.0755754496166952
P1: 0.22891044301706723
P2: 0.3488110513492265
P3: 0.346703056017011
Q: 0.6609259259259259
S: 0.9913888888888889
P_reject: 0.3385185185185185
K: 1.9666417137665533
arrivals: 5400
served: 3569
rejected: 1828


In [8]:
# load increase
print("\nLoad scaling (exponential arrivals)")
for factor in [2,3,4]:
    res = simulate(lambda: exp_interarrival(1.5*factor), lambda: exp_service(2.0))
    print(f"\nLambda x{factor}")
    for k,v in res.items():
        print(f"{k}: {v}")


Load scaling (exponential arrivals)

Lambda x2
P0: 0.016247201792354053
P1: 0.09831723296235974
P2: 0.2959037047155333
P3: 0.5895318605297529
Q: 0.4149986163638041
S: 1.2497222222222222
P_reject: 0.5847246563970113
K: 2.4587202239826853
arrivals: 10841
served: 4499
rejected: 6339

Lambda x3
P0: 0.007984009210254288
P1: 0.05722595666608612
P2: 0.24150655215543035
P3: 0.6932834819682292
Q: 0.30262172284644195
S: 1.3466666666666667
P_reject: 0.6971910112359551
K: 2.6200895068816346
arrivals: 16020
served: 4848
rejected: 11169

Lambda x4
P0: 0.003163497281523883
P1: 0.034841196717079796
P2: 0.19214960651305524
P3: 0.769845699488341
Q: 0.22649712495909494
S: 1.3458333333333334
P_reject: 0.7733626291430976
K: 2.728677508208213
arrivals: 21391
served: 4845
rejected: 16543


In [14]:
# IS structure augmentation
# addition of a 4th server
def sim_servers(n_servers, arrival_func, service_func, sim_time=SIM_TIME):
    time = 0.0
    busy = 0
    departures = []

    arrivals = served = rejected = 0
    state_time = defaultdict(float)
    last_event_time = 0.0
    next_arrival = arrival_func()

    while time < sim_time:
        next_departure = departures[0] if departures else math.inf

        if next_arrival <= next_departure:
            time = next_arrival
            state_time[busy] += time - last_event_time
            last_event_time = time
            
            arrivals += 1
            if busy < n_servers:
                busy += 1
                heapq.heappush(departures, time + service_func())
            else:
                rejected += 1
            
            next_arrival = time + arrival_func()
        else:
            time = next_departure
            state_time[busy] += time - last_event_time
            last_event_time = time
            busy -= 1
            served += 1
            heapq.heappop(departures)

    total_time = sum(state_time.values())
    P = {k: state_time[k] / total_time for k in range(n_servers + 1)}
    Q = served / arrivals if arrivals else 0
    S = served / sim_time
    P_reject = rejected / arrivals if arrivals else 0
    K = sum(k*P[k] for k in range(n_servers + 1))

    return {'Q': Q, 'S': S, 'P_reject': P_reject, 'K': K}

print("IS structure comparison")
base = sim_servers(3, lambda:exp_interarrival(1.5), lambda: exp_service(2.0))
upgraded = sim_servers(4, lambda:exp_interarrival(1.5), lambda: exp_service(2.0))

print("\n3 servers:")
for k,v in base.items():
    print(f"{k}: {v}")
print("\n4 servers:")
for k,v in upgraded.items():
    print(f"{k}: {v}")



IS structure comparison

3 servers:
Q: 0.6604155175583747
S: 0.9977777777777778
P_reject: 0.3390329104614819
K: 1.9672237640302574

4 servers:
Q: 0.7889670492410218
S: 1.183888888888889
P_reject: 0.21084783413550537
K: 2.395504797768508
